# Sprint 5 Review

## Our Project

### Group
The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.


### Individual
My individual feature focuses on implementing the real-time guessing system. This feature enables players to submit guesses while others draw, providing instant feedback on whether the guess is correct. The purpose of this feature is to ensure smooth and engaging gameplay by allowing seamless interaction between players and keeping the game dynamic and competitive.

## FullStack Features

Live Demo

## List Requests

In our program, we make use of lists, dictionaries, and a database to handle the application's data. Specifically:

- Lists are used to store rows of guesses fetched from the database, where each row corresponds to an individual guess.
- Dictionaries are used to represent columns in the database for each row, allowing easy access to data fields like guesser_name, guess, and is_correct.
- Database (SQLite) is used to persistently store guesses, ensuring data integrity and retrieval even after the application is restarted.

### List Code descriptions
- Lists: In the submit_guesses API, we query the SQLite database to fetch all guesses for a specific user. The query returns a list of rows, with each row corresponding to an individual guess.

- Descriptions: We use the READ method of the Guess class to convert a row into a dictionary. Each dictionary contains key-value pairs representing column names and their respective values.

- Formatting Response Data (JSON) from API into DOM: 
When a user interacts with the frontend, the backend sends data in JSON format. For example, when fetching guesses for a user, the API response looks like this:

[
    {

        "guesser_name": "JohnDoe",
        "guess": "apple",
        "is_correct": true
    },
    {
        "guesser_name": "JohnDoe",
        "guess": "banana",
        "is_correct": false
    }
]

- Database Queries Extracting Python Lists (Rows):
SQLite, managed through SQLAlchemy, is the third-party library used in our program. It retrieves a list of Guess objects (rows) matching the condition guesser_name=user. 

- Methods in Class to Work with Columns:
    - Create: Adds a new row (guess) to the database.
    def create(self):
    db.session.add(self)
    db.session.commit()
    
    - Read: Converts a row into a dictionary for JSON responses.
    def read(self):
    return {
        "id": self.id,
        "guesser_name": self.guesser_name,
        "guess": self.guess,
        "is_correct": self.is_correct
    }

    - Update: Modifies the fields of an existing guess and commits the changes.
    def update(self, data):
    for key, value in data.items():
        setattr(self, key, value)
    db.session.commit()

    - Delete: Removes a row from the database.
    def delete(self):
    db.session.delete(self)
    db.session.commit()



## Algorithmic code request.

### Definition of Code Blocks to Handle Requests

The API class in our program handles HTTP methods (GET, POST, PUT, and DELETE) for interacting with guesses in the database. Each method is mapped to a specific route and contains logic to process the request and return an appropriate response.

### API Class and Methods for CRUD Operations

- GET Method: Retrieve Guesses:
    The GET method allows users to fetch guesses from the database.

    @app.route('/api/guesses', methods=['GET'])
def get_guesses():
    try:
        # Extract the 'user' query parameter
        user = request.args.get('user', type=str)

        if user:
            # Query the database for guesses by the specified user
            guesses = Guess.query.filter_by(guesser_name=user).all()
            if not guesses:
                return jsonify({"error": "No guesses found for this user."}), 404
            return jsonify([guess.read() for guess in guesses]), 200
        else:
            return jsonify({"error": "Please provide the 'user' as a query parameter."}), 400

    except Exception as e:
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500




- POST Method: Add a New Guess;
    The POST method adds a new guess to the database.

    @app.route('/api/guesses', methods=['POST'])
def add_guess():
    try:
        data = request.json
        if not data:
            return jsonify({"error": "Invalid or missing JSON payload."}), 400

        # Extract data from request
        guesser_name = data.get("guesser_name")
        guess = data.get("guess")
        is_correct = data.get("is_correct")

        # Validate required fields
        if not all([guesser_name, guess, is_correct is not None]):
            return jsonify({"error": "Missing required fields."}), 400

        # Create and save a new guess
        new_guess = Guess(guesser_name=guesser_name, guess=guess, is_correct=is_correct)
        new_guess.create()

        return jsonify(new_guess.read()), 201
    except Exception as e:
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500



- PUT Method: Update an Existing Guess
    The PUT method updates specific fields of a guess.

    @app.route('/api/guesses', methods=['PUT'])
def update_guess():
    try:
        data = request.json
        if not data:
            return jsonify({"error": "Invalid or missing JSON payload."}), 400

        guesser_name = data.get("guesser_name")
        if not guesser_name:
            return jsonify({"error": "Missing 'guesser_name' in payload."}), 400

        # Query the database for the user's latest guess
        guess = Guess.query.filter_by(guesser_name=guesser_name).order_by(Guess.id.desc()).first()
        if not guess:
            return jsonify({"error": "Guess not found for the specified user."}), 404

        # Update fields and save
        guess.update(data)
        return jsonify(guess.read()), 200
    except Exception as e:
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500



- DELETE Method: Remove a Guess
The DELETE method removes a guess from the database.

@app.route('/api/guesses', methods=['DELETE'])
def delete_guess():
    try:
        data = request.json
        if not data:
            return jsonify({"error": "Invalid or missing JSON payload."}), 400

        guesser_name = data.get("guesser_name")
        if not guesser_name:
            return jsonify({"error": "Missing 'guesser_name' in payload."}), 400

        # Query the database for the user's latest guess
        guess = Guess.query.filter_by(guesser_name=guesser_name).order_by(Guess.id.desc()).first()
        if not guess:
            return jsonify({"error": "Guess not found for the specified user."}), 404

        # Delete the guess
        guess.delete()
        return jsonify({"message": "Guess deleted successfully."}), 200
    except Exception as e:
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500




### Method/Procedure with Sequencing, Selection, and Iteration

The add_guess (POST method) contains all three components:

- Sequencing: The method performs steps in order: extracting data, validating input, and saving to the database.

- Selection: The if not data and if not all([...]) checks validate required fields.

- Iteration: Not explicitly shown in this method, but iteration occurs when converting query results to JSON ([guess.read() for guess in guesses]).

### Parameters and Return Type

- Parameters
    The body of the request is in JSON format, which contains fields like:

        guesser_name: Name of the guesser (string).
        guess: The guessed value (string).
        is_correct: Whether the guess is correct (boolean).

- Return Type
The functions return a JSON response, created using jsonify(), which ensures the response is in proper JSON format. 
     
     {
    "id": 1,
    "guesser_name": "JohnDoe",
    "guess": "apple",
    "is_correct": true
}


## Call to Algorithm request

### Definition of Code Block to Make a Request

To make a request to the backend, we use the fetch API in JavaScript for the client-side. Below is an example of a fetch call to the /api/guesses endpoint to retrieve, add, update, or delete data.

ex: 
const getGuessesForUser = async (username) => {
  try {
    const response = await fetch(`/api/guesses?user=${username}`, {
      method: 'GET',
      headers: {
        'Content-Type': 'application/json',
      },
    });

    if (response.ok) {
      const data = await response.json();
      console.log("User's guesses:", data);
      return data;
    } else {
      const errorData = await response.json();
      console.error('Error fetching guesses:', errorData);
      return errorData;
    }
  } catch (error) {
    console.error('Network or server error:', error);
  }
};


### 
Here’s a detailed breakdown of a Call to Algorithm request:

Definition of Code Block to Make a Request
To make a request to the backend, we use the fetch API in JavaScript for the client-side. Below is an example of a fetch call to the /api/guesses endpoint to retrieve, add, update, or delete data.

Code Block (Example: Fetching guesses for a specific user):

javascript
Copy
Edit
const getGuessesForUser = async (username) => {
  try {
    const response = await fetch(`/api/guesses?user=${username}`, {
      method: 'GET',
      headers: {
        'Content-Type': 'application/json',
      },
    });

    if (response.ok) {
      const data = await response.json();
      console.log("User's guesses:", data);
      return data;
    } else {
      const errorData = await response.json();
      console.error('Error fetching guesses:', errorData);
      return errorData;
    }
  } catch (error) {
    console.error('Network or server error:', error);
  }
};




### Call/Request to the Method with Algorithm

The frontend sends a POST request to the /api/guesses endpoint to add a new guess to the database. The payload is sent as JSON.

ex: 
const addNewGuess = async (guesserName, guess, isCorrect) => {
  try {
    const response = await fetch('/api/guesses', {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
      },
      body: JSON.stringify({
        guesser_name: guesserName,
        guess: guess,
        is_correct: isCorrect,
      }),
    });

    if (response.ok) {
      const data = await response.json();
      console.log('Guess added successfully:', data);
      return data;
    } else {
      const errorData = await response.json();
      console.error('Error adding guess:', errorData);
      return errorData;
    }
  } catch (error) {
    console.error('Network or server error:', error);
  }
};


### Return/Response from the Method with Algorithm

- Successful Response Handling:
    When the request is successful, the backend responds with a JSON object containing the added or retrieved data. For example: POST Success Response:  

    {
  "id": 3,
  "guesser_name": "JohnDoe",
  "guess": "apple",
  "is_correct": true
}

- Frontend Handling: The fetch function receives the response and logs it or updates the DOM to display the new data:

addNewGuess('JohnDoe', 'apple', true).then((data) => {
  if (data.id) {
    console.log('New guess added with ID:', data.id);
    updateGuessListOnPage(data); // Example function to update UI
  }
});

- Error Response Handling:
If there’s an error in the request or the backend logic, the response will include an error message: POST Error Response:

{
  "error": "Missing required fields."
}

- Frontend Handling: The error is logged, and an appropriate message is displayed to the user:

addNewGuess('', '', true).then((data) => {
  if (data.error) {
    console.error('Error:', data.error);
    showErrorMessageOnPage(data.error); // Example function to update UI
  }
});

- Changing Data or Method Triggers Different Responses
ex: 
Input: 
addNewGuess('JaneDoe', '42', true); 

Response:

{
  "id": 4,
  "guesser_name": "JaneDoe",
  "guess": "42",
  "is_correct": true
}

Frontend Handling: 

const handleGuessSubmission = async () => {
  const result = await addNewGuess('JaneDoe', '42', true);
  if (result.error) {
    showErrorMessageOnPage(result.error); // Display error
  } else {
    updateGuessListOnPage(result); // Add success to DOM
  }
};


